In [1]:
import mlflow
import mlflow.sklearn 
import joblib 
from mlflow.tracking import MlflowClient

In [2]:
mlflow.set_tracking_uri("http://localhost:5080")
mlflow.set_experiment("forex_direction_rf")

MODEL_NAME = "eur_usd_forex_direction_model"

model = joblib.load("model_service/forex_model.pkl")

with mlflow.start_run(run_name="initial_champion"):
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="model",
        registered_model_name=MODEL_NAME
    )

    run_id = mlflow.active_run().info.run_id

client = MlflowClient()

latest_versions = client.get_latest_versions(
    MODEL_NAME, stages=["None"]
)

model_version = latest_versions[0].version

client.transition_model_version_stage(
    name=MODEL_NAME,
    version=model_version,
    stage="Production",
    archive_existing_versions=True
)

print(f"Model {MODEL_NAME} v{model_version} promoted to Production")


2026/01/08 11:22:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/08 11:22:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'eur_usd_forex_direction_model'.
2026/01/08 11:22:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: eur_usd_forex_direction_model, version 1


🏃 View run initial_champion at: http://localhost:5080/#/experiments/1/runs/663ac199fd404565b448ec84a8b9a816
🧪 View experiment at: http://localhost:5080/#/experiments/1
Model eur_usd_forex_direction_model v1 promoted to Production


Created version '1' of model 'eur_usd_forex_direction_model'.
/var/folders/5l/17wd4myj387b056cpgkkpxth0000gn/T/ipykernel_10113/693387172.py:19: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(
/var/folders/5l/17wd4myj387b056cpgkkpxth0000gn/T/ipykernel_10113/693387172.py:25: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [2]:
import mlflow
import mlflow.sklearn 
import joblib 
from mlflow.tracking import MlflowClient

In [3]:
import mlflow.pyfunc

mlflow.set_tracking_uri("http://localhost:5080")
MODEL_URI = "models:/eur_usd_forex_direction_model/Production"

model = mlflow.pyfunc.load_model(MODEL_URI)

In [4]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: mlflow-artifacts:/1/models/m-09f6deabd533433284979c64648c18eb/artifacts
  flavor: mlflow.sklearn
  run_id: 663ac199fd404565b448ec84a8b9a816

In [5]:
model.model_id

'm-09f6deabd533433284979c64648c18eb'

In [6]:
model._model_meta

In [11]:
model.metadata.model_name

AttributeError: 'Model' object has no attribute 'model_name'

In [10]:
client = MlflowClient()

run_id = model.metadata.run_id

for rm in client.search_registered_models():
    for mv in rm.latest_versions:
        if mv.run_id == run_id:
            print("Model name:", rm.name)
            print("Version:", mv.version)
            print("Stage:", mv.current_stage)

Model name: eur_usd_forex_direction_model
Version: 1
Stage: Production
